In [ ]:
#BanglaLLM/Bangla-s1k-llama-3.2-3B-Instruct,
#BanglaLLM/Bangla-s1k-qwen-2.5-3B-Instruct,
#BanglaLLM/BanglaLLama-3.2-3b-bangla-alpaca-orca-instruct-v0.0.1,
#BanglaLLM/BanglaLLama-3.2-3b-unolp-culturax-base-v0.0.3

In [ ]:
# ✅ Mount Google Drive
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# ✅ Install Required Libraries
!pip install -q transformers datasets accelerate bitsandbytes peft huggingface_hub torch==2.6.0 unsloth unsloth_zoo


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 47.1/47.1 kB 3.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 1.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 120.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 24.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 55.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 7.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 14.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 8.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 1.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 47.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.0/67.0 MB 11.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [ ]:
# ✅ Set CUDA Memory Optimization
import os
os.environ["PYTORCH_CUDA_ALLOC_CONF"] = "expandable_segments:True"


In [ ]:
# ✅ Define Paths
PROJECT_DIR = "/content/drive/MyDrive/BanglaLLM_FakeReview"
MODEL_OUTPUT_DIR = f"{PROJECT_DIR}/llm_model_outputs"
FINAL_MODEL_DIR = f"{PROJECT_DIR}/final_banglallm_model"
os.makedirs(MODEL_OUTPUT_DIR, exist_ok=True)
os.makedirs(FINAL_MODEL_DIR, exist_ok=True)

In [ ]:
# ✅ Load Dataset and Balance
from datasets import load_dataset
import random

In [ ]:
# Load and split
dataset = load_dataset("shawon95/Bengali-Fake-Review-Dataset")
if "test" not in dataset:
    dataset = dataset["train"].train_test_split(test_size=0.2, seed=42)


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md:   0%|          | 0.00/3.14k [00:00<?, ?B/s]

fake.csv:   0%|          | 0.00/1.90M [00:00<?, ?B/s]

non-fake.csv:   0%|          | 0.00/12.1M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/9049 [00:00<?, ? examples/s]

In [ ]:
from datasets import concatenate_datasets

# ✅ Balance training data safely
fake = dataset["train"].filter(lambda x: x["Label"] == 1)
non_fake = dataset["train"].filter(lambda x: x["Label"] == 0)

min_class_size = min(len(fake), len(non_fake))  # Find the smaller class size

fake_sample = fake.shuffle(seed=42).select(range(min_class_size))
non_fake_sample = non_fake.shuffle(seed=42).select(range(min_class_size))

from datasets import concatenate_datasets
balanced_train = concatenate_datasets([fake_sample, non_fake_sample])
balanced_train = balanced_train.shuffle(seed=42)

dataset["train"] = balanced_train

print(f"✅ Balanced dataset created with {len(balanced_train)} samples")


✅ Balanced dataset created with 2112 samples


In [ ]:
from datasets import concatenate_datasets

# ✅ Balance test data
fake_test = dataset["test"].filter(lambda x: x["Label"] == 1)
non_fake_test = dataset["test"].filter(lambda x: x["Label"] == 0)

min_test_size = min(len(fake_test), len(non_fake_test))
fake_test = fake_test.shuffle(seed=42).select(range(min_test_size))
non_fake_test = non_fake_test.shuffle(seed=42).select(range(min_test_size))

balanced_test = concatenate_datasets([fake_test, non_fake_test]).shuffle(seed=42)
dataset["test"] = balanced_test

print(f"✅ Balanced test set size: {len(dataset['test'])}")


Filter:   0%|          | 0/566 [00:00<?, ? examples/s]

Filter:   0%|          | 0/566 [00:00<?, ? examples/s]

✅ Balanced test set size: 566


In [ ]:
# ✅ Tokenization
from transformers import AutoTokenizer

MODEL_NAME = "BanglaLLM/Bangla-s1k-llama-3.2-3B-Instruct"
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)
tokenizer.pad_token = tokenizer.eos_token

def tokenize_function(examples):
    tokens = tokenizer(examples["Review"], padding="max_length", truncation=True, max_length=256)
    tokens["labels"] = tokens["input_ids"].copy()
    return tokens

from datasets import DatasetDict

tokenized_train = dataset["train"].map(tokenize_function, batched=True)
tokenized_test = dataset["test"].map(tokenize_function, batched=True)

tokenized_train = tokenized_train.remove_columns(["Review", "Label"])
tokenized_test = tokenized_test.remove_columns(["Review", "Label"])

tokenizer_config.json:   0%|          | 0.00/54.6k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.2M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/343 [00:00<?, ?B/s]

Map:   0%|          | 0/2112 [00:00<?, ? examples/s]

Map:   0%|          | 0/1810 [00:00<?, ? examples/s]

In [ ]:
# ✅ Load Model with Unsloth
from unsloth import FastLanguageModel
import torch

model, tokenizer = FastLanguageModel.from_pretrained(
    model_name=MODEL_NAME,
    max_seq_length=512,
    dtype=torch.float16,
    load_in_4bit=True
)

tokenizer.pad_token = tokenizer.eos_token

model = FastLanguageModel.get_peft_model(
    model,
    r=16,
    lora_alpha=32,
    lora_dropout=0.05,
    bias="none",
    use_gradient_checkpointing=True,
    random_state=42,
    use_rslora=False,
    loftq_config=None
)

model.print_trainable_parameters()

<ipython-input-11-2134140900>:2: UserWarning: WARNING: Unsloth should be imported before transformers to ensure all optimizations are applied. Your code may run slower or encounter memory issues without these optimizations.

Please restructure your imports with 'import unsloth' at the top of your file.
  from unsloth import FastLanguageModel


🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.
🦥 Unsloth Zoo will now patch everything to make training faster!
==((====))==  Unsloth 2025.6.2: Fast Llama patching. Transformers: 4.52.4.
   \\   /|    Tesla T4. Num GPUs = 1. Max memory: 14.741 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.6.0+cu124. CUDA: 7.5. CUDA Toolkit: 12.4. Triton: 3.2.0
\        /    Bfloat16 = FALSE. FA [Xformers = 0.0.29.post3. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


model.safetensors.index.json:   0%|          | 0.00/21.0k [00:00<?, ?B/s]

Fetching 3 files:   0%|          | 0/3 [00:00<?, ?it/s]

model-00001-of-00003.safetensors:   0%|          | 0.00/5.00G [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/54.6k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.2M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/343 [00:00<?, ?B/s]

Unsloth: Dropout = 0 is supported for fast patching. You are using dropout = 0.05.
Unsloth will patch all other layers, except LoRA matrices, causing a performance hit.
Unsloth 2025.6.2 patched 28 layers with 0 QKV layers, 0 O layers and 0 MLP layers.


trainable params: 24,313,856 || all params: 3,237,063,680 || trainable%: 0.7511


In [ ]:
from transformers import TrainingArguments

training_args = TrainingArguments(
    output_dir=MODEL_OUTPUT_DIR,
    save_steps=100,                   # ✅ Save every N steps (e.g., 100)
    eval_steps=100,                   # ✅ Evaluate every N steps
    save_total_limit=3,
    learning_rate=2e-4,
    per_device_train_batch_size=4,
    per_device_eval_batch_size=4,
    gradient_accumulation_steps=8,
    num_train_epochs=2,
    weight_decay=0.01,
    logging_steps=10,
    fp16=True,
    report_to="none",
    remove_unused_columns=False,
    load_best_model_at_end=False      # ✅ Turn this off to avoid dependency on matching strategies
)


In [ ]:
# ✅ Resume from checkpoint if available
last_checkpoint = None
if os.path.isdir(MODEL_OUTPUT_DIR):
    checkpoints = [f.path for f in os.scandir(MODEL_OUTPUT_DIR) if f.is_dir() and "checkpoint" in f.name]
    if checkpoints:
        last_checkpoint = sorted(checkpoints)[-1]

print("🔁 Resuming from:", last_checkpoint if last_checkpoint else "scratch")

🔁 Resuming from: scratch


In [ ]:
# ✅ Train Model
from transformers import Trainer

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_train,
    eval_dataset=tokenized_test,
    tokenizer=tokenizer,
    data_collator=None
)

trainer.train(resume_from_checkpoint=last_checkpoint)

<ipython-input-16-4114786203>:4: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(
==((====))==  Unsloth - 2x faster free finetuning | Num GPUs used = 1
   \\   /|    Num examples = 2,112 | Num Epochs = 2 | Total steps = 132
O^O/ \_/ \    Batch size per device = 4 | Gradient accumulation steps = 8
\        /    Data Parallel GPUs = 1 | Total batch size (4 x 8 x 1) = 32
 "-____-"     Trainable parameters = 24,313,856/3,000,000,000 (0.81% trained)


Unsloth: Will smartly offload gradients to save VRAM!


Step,Training Loss
10,1.748100
20,1.538200
30,1.415500
40,1.373400
50,1.323500
60,1.325000
70,1.279200
80,1.250800
90,1.188900
100,1.192700


TrainOutput(global_step=132, training_loss=1.3254300301725215, metrics={'train_runtime': 1594.0734, 'train_samples_per_second': 2.65, 'train_steps_per_second': 0.083, 'total_flos': 1.844596333294387e+16, 'train_loss': 1.3254300301725215, 'epoch': 2.0})

In [ ]:
# ✅ Save Final Model & Tokenizer
trainer.save_model(FINAL_MODEL_DIR)
tokenizer.save_pretrained(FINAL_MODEL_DIR)


('/content/drive/MyDrive/BanglaLLM_FakeReview/final_banglallm_model/tokenizer_config.json',
 '/content/drive/MyDrive/BanglaLLM_FakeReview/final_banglallm_model/special_tokens_map.json',
 '/content/drive/MyDrive/BanglaLLM_FakeReview/final_banglallm_model/chat_template.jinja',
 '/content/drive/MyDrive/BanglaLLM_FakeReview/final_banglallm_model/tokenizer.json')

In [ ]:
# ✅ Final Evaluation + Softmax with robust decoding (improved matching logic)
from sklearn.metrics import classification_report, precision_recall_fscore_support, accuracy_score
import torch
import json
import transformers

transformers.logging.set_verbosity_error()

true_labels, pred_labels, softmax_probs = [], [], []
model.eval()

for example in dataset["test"]:
    review_text = example["Review"][:300]  # truncate long reviews
    true_label = example["Label"]

    prompt = f"""তুমি একজন রিভিউ বিশ্লেষক। নিচের রিভিউটি পড়ে শুধুমাত্র একটি শব্দে উত্তর দাও: 'ভুয়া' বা 'সত্যিকারের'।\n\nরিভিউ:\n{review_text}\n\nউত্তর:"""

    inputs = tokenizer(prompt, return_tensors="pt", truncation=True, padding=True, max_length=512).to("cuda")
    with torch.no_grad():
        output = model.generate(
            **inputs,
            max_new_tokens=10,
            do_sample=False,
            num_beams=1
        )

    decoded = tokenizer.decode(output[0], skip_special_tokens=True).lower().strip()

    # ✅ Robust post-processing
    if "উত্তর:" in decoded:
        decoded = decoded.split("উত্তর:")[-1].strip()
    elif ":" in decoded:
        decoded = decoded.split(":")[-1].strip()

    decoded = decoded.replace("।", "").replace(".", "").strip()
    decoded = decoded[:20]  # cut garbage or extra tokens

    # ✅ Classify by keyword (improved logic)
    if any(x in decoded for x in ["ভুয়া", "fake", "নকল", "বাজে", "ভয়ংকর", "জাল"]):
        pred = 1
    elif any(x in decoded for x in ["সত্যিকারের", "সত্যিকার", "real", "আসল", "original", "ভালো", "good"]):
        pred = 0
    else:
        pred = 1

    pred_labels.append(pred)
    true_labels.append(true_label)
    softmax_probs.append({"text": review_text, "decoded": decoded, "label": true_label, "pred": pred})

# ✅ Save decoded predictions
with open(f"{PROJECT_DIR}/llama_s1k_decoded_predictions.json", "w") as f:
    json.dump(softmax_probs, f)

# ✅ Show final metrics
accuracy = accuracy_score(true_labels, pred_labels)
precision, recall, f1, _ = precision_recall_fscore_support(true_labels, pred_labels, average='weighted', zero_division=0)

print(f"\n🔹 Accuracy: {accuracy:.4f}")
print(f"🔹 Precision: {precision:.4f}")
print(f"🔹 Recall: {recall:.4f}")
print(f"🔹 F1-Score: {f1:.4f}")

print("\n🔎 Detailed Classification Report:")
print(classification_report(true_labels, pred_labels, target_names=["Fake", "Non-Fake"], digits=4))
print(f"\n✅ Total samples processed: {len(true_labels)} (should match test dataset)")



🔹 Accuracy: 0.5124
🔹 Precision: 0.5233
🔹 Recall: 0.5124
🔹 F1-Score: 0.4475

🔎 Detailed Classification Report:
              precision    recall  f1-score   support

        Fake     0.5393    0.1696    0.2581       283
    Non-Fake     0.5073    0.8551    0.6368       283

    accuracy                         0.5124       566
   macro avg     0.5233    0.5124    0.4475       566
weighted avg     0.5233    0.5124    0.4475       566


✅ Total samples processed: 566 (should match test dataset)


In [ ]:
# ✅ Improved Final Evaluation with enhanced prompt & decoding
from sklearn.metrics import classification_report, precision_recall_fscore_support, accuracy_score
import torch
import json
import re
import transformers

transformers.logging.set_verbosity_error()

true_labels, pred_labels, softmax_probs = [], [], []
model.eval()

for example in dataset["test"]:
    review_text = example["Review"][:300]
    true_label = example["Label"]

    # ✅ Concise and instruction-optimized prompt
    prompt = f"এই রিভিউটি ভুয়া না সত্যিকারের? এক শব্দে উত্তর দাও:\n\nরিভিউ:\n{review_text}\n\nউত্তর:"

    inputs = tokenizer(prompt, return_tensors="pt", truncation=True, padding=True, max_length=512).to("cuda")
    with torch.no_grad():
        output = model.generate(
            **inputs,
            max_new_tokens=15,
            do_sample=False,
            num_beams=1,
            temperature=0.7,
            repetition_penalty=1.2
        )

    decoded = tokenizer.decode(output[0], skip_special_tokens=True).lower().strip()

    # ✅ Post-process: clean & slice
    decoded = re.sub(r"[।:]", "", decoded)
    decoded = decoded.strip()[:20]

    # ✅ Classification logic (expanded match)
    if any(x in decoded for x in ["ভুয়া", "fake", "নকল", "জাল", "মিথ্যা"]):
        pred = 1
    elif any(x in decoded for x in ["সত্যিকারের", "সত্যিকার", "আসল", "original", "real", "বাস্তব"]):
        pred = 0
    else:
        pred = 1  # fallback to fake

    pred_labels.append(pred)
    true_labels.append(true_label)
    softmax_probs.append({
        "text": review_text,
        "decoded": decoded,
        "label": true_label,
        "pred": pred
    })

# ✅ Save & Report
with open(f"{PROJECT_DIR}/llama_s1k_final_predictions.json", "w") as f:
    json.dump(softmax_probs, f)

accuracy = accuracy_score(true_labels, pred_labels)
precision, recall, f1, _ = precision_recall_fscore_support(true_labels, pred_labels, average='weighted', zero_division=0)

print(f"\n🔹 Accuracy: {accuracy:.4f}")
print(f"🔹 Precision: {precision:.4f}")
print(f"🔹 Recall: {recall:.4f}")
print(f"🔹 F1-Score: {f1:.4f}")
print("\n🔎 Detailed Report:")
print(classification_report(true_labels, pred_labels, target_names=["Fake", "Non-Fake"], digits=4))



🔹 Accuracy: 0.5000
🔹 Precision: 0.2500
🔹 Recall: 0.5000
🔹 F1-Score: 0.3333

🔎 Detailed Report:
              precision    recall  f1-score   support

        Fake     0.0000    0.0000    0.0000       283
    Non-Fake     0.5000    1.0000    0.6667       283

    accuracy                         0.5000       566
   macro avg     0.2500    0.5000    0.3333       566
weighted avg     0.2500    0.5000    0.3333       566



/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


In [ ]:
print(f"🧾 Prompt: {prompt}")
print(f"🧾 Decoded: {decoded}")


🧾 Prompt: তুমি একজন রিভিউ বিশ্লেষক। নিচের রিভিউটি পড়ে শুধুমাত্র একটি শব্দে উত্তর দাও: 'ভুয়া' বা 'সত্যিকারের'।

রিভিউ:
#৪৯ টাকার অফার কালকেই শেষ দিন তাই আজকে
#৪৯টাকা দিয়ে মন ভরে খেয়ে আসলাম!!
অর্ডার করছিলামঃ
😍থাই সুপ-৪৯টাকা
😍বারবিকিউ উইন্সঃ৩ পিস-৪৯টাকা
😍বিফ বার্গার -৯৯টাকা
💝থাই স্যুপ এর ভিতরে মাশরুম চিকেন মুটামুটি ৪৯ টাকা হিসাবে ভালই ছিল! 
💝উইংস এর কথা আর কি বলব ৪৯ টাকায় ৩ পিস উইংস আসলেই অসম্ভব টেস্ট জোস।
💝বার্গার ট

উত্তর:
🧾 Decoded: ভুয়া


In [ ]:
import json
with open(f"{PROJECT_DIR}/llama_s1k_decoded_predictions.json") as f:
    data = json.load(f)

for d in data[:10]:
    print("✅ Text:", d["text"][:50])
    print("🔍 Decoded:", d["decoded"])
    print("🎯 Pred:", d["pred"])
    print("🟢 True Label:", d["label"])
    print("-----")


✅ Text: বড় বড় ক্ষুধার এক বিশাল সমাধান। 
খিলগাঁও গেলাম আজ ত
🔍 Decoded: সত্যিকার
🎯 Pred: 0
🟢 True Label: 0
-----
✅ Text: আমার খাওয়া সবথেকে বেস্ট পাস্তা..এত্ত চিজ আর চিকেন 
🔍 Decoded: ভুয়া
🎯 Pred: 1
🟢 True Label: 1
-----
✅ Text: #সব_খাবার_একসাথেবেস্ট ফাস্ট ফুড এক্সপেরিয়েন্স হয়েছ
🔍 Decoded: ভুয়া
🎯 Pred: 1
🟢 True Label: 1
-----
✅ Text: আইটেম: ছোলা বাটোরাটেস্ট: চাটনি,  সাদা সস আর ডালটা 
🔍 Decoded: ভুয়া
🎯 Pred: 1
🟢 True Label: 1
-----
✅ Text: জীবনের প্রথম বার পরিবারকে নিয়ে এতো কম টাকায় উন্নত 
🔍 Decoded: ভুয়া
🎯 Pred: 1
🟢 True Label: 1
-----
✅ Text: মিরপুর-10 এ পেয়ে গেলাম এক অসাধারণ প্যাকেজ। ওভারঅল 
🔍 Decoded: সত্যিকার
🎯 Pred: 0
🟢 True Label: 1
-----
✅ Text: ৬টা আইটেমের ফ্রেন্ডস প্ল্যাটার মাত্র ২৫০/- (পার পা
🔍 Decoded: ভুয়া
🎯 Pred: 1
🟢 True Label: 1
-----
✅ Text: # কাবাব _এক্সপেরিয়েন্স _করার_ _জন্য _খিলগাঁও_ The 
🔍 Decoded: সত্যিকার
🎯 Pred: 0
🟢 True Label: 1
-----
✅ Text: মিরপুর নদীতে যাওয়ার আগে ছোট বড় নদী পার হয়ে আসতে হয়
🔍 Decoded: ভুয়া
🎯 Pred: 1
🟢 True Label: 0
-----
✅ Text: বফের দুইটা 